# Applying LSTM Models on Raw Data

In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

    
    result = confusion_matrix(Y_true, Y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(result, 
                xticklabels= list(ACTIVITIES.values()), 
                yticklabels=list(ACTIVITIES.values()), 
                annot=True, fmt="d");
    plt.title("Confusion matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show() 

### Loading Data

In [ ]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [ ]:
# define a function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/MyDrive/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [ ]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [ ]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, y_train, y_test = load_data()

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


 1. Defining the Architecture of 1-Layer of LSTM

In [ ]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 12s 9ms/step - loss: 1.1016 - accuracy: 0.5365 - val_loss: 0.8723 - val_accuracy: 0.6138
Epoch 2/30
460/460 [==============================] - 5s 11ms/step - loss: 0.7845 - accuracy: 0.6489 - val_loss: 0.7582 - val_accuracy: 0.6766
Epoch 3/30
460/460 [==============================] - 4s 8ms/step - loss: 0.6789 - accuracy: 0.6952 - val_loss: 0.6788 - val_accuracy: 0.7248
Epoch 4/30
460/460 [==============================] - 3s 8ms/step - loss: 0.5698 - accuracy: 0.7821 - val_loss: 0.5663 - val_accuracy: 0.8032
Epoch 5/30
460/460 [==============================] - 4s 8ms/step - loss: 0.4379 - accuracy: 0.8619 - val_loss: 0.5142 - val_accuracy: 0.8286
Epoch 6/30
460/460 [==============================] - 6s 12ms/step - loss: 0.3452 - accuracy: 0.8983 - val_loss: 0.4806 - val_accuracy: 0.8371
Epoch 7/30
460/460 [==============================] - 5s 11ms/step - loss: 0.2986 - accuracy: 0.9119 - val_loss: 0.5096 - val_accuracy: 0.8521
Ep

In [ ]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 1s 4ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,2,25,0,0
SITTING,2,367,117,3,0,2
STANDING,0,66,464,2,0,0
WALKING,0,0,0,451,11,34
WALKING_DOWNSTAIRS,0,0,0,1,400,19
WALKING_UPSTAIRS,0,1,0,2,0,468


In [ ]:
score = model.evaluate(X_test, y_test)

print("\n   categorical_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 0s 4ms/step - loss: 0.4404 - accuracy: 0.9026

   categorical_crossentropy  ||   accuracy 
  ____________________________________
[0.44041481614112854, 0.9026128053665161]


   2. Defining the Architecture of 2-Layer of LSTM with more hyperparameter tunning

#### 2.1 First Model for 2-Layer of LSTM with more hyperparameter tunning

In [ ]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization (BatchN  (None, 128, 48)          192       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 12s 17ms/step - loss: 1.5451 - accuracy: 0.6557 - val_loss: 2.1739 - val_accuracy: 0.5616
Epoch 2/30
460/460 [==============================] - 6s 14ms/step - loss: 0.8112 - accuracy: 0.8226 - val_loss: 0.5294 - val_accuracy: 0.8802
Epoch 3/30
460/460 [==============================] - 7s 15ms/step - loss: 0.4346 - accuracy: 0.8973 - val_loss: 0.3874 - val_accuracy: 0.8609
Epoch 4/30
460/460 [==============================] - 6s 14ms/step - loss: 0.2517 - accuracy: 0.9238 - val_loss: 0.2453 - val_accuracy: 0.9145
Epoch 5/30
460/460 [==============================] - 7s 16ms/step - loss: 0.2279 - accuracy: 0.9260 - val_loss: 0.2249 - val_accuracy: 0.9104
Epoch 6/30
460/460 [==============================] - 7s 14ms/step - loss: 0.2111 - accuracy: 0.9305 - val_loss: 0.2056 - val_accuracy: 0.9264
Epoch 7/30
460/460 [==============================] - 7s 16ms/step - loss: 0.1842 - accuracy: 0.9370 - val_loss: 0.2080 - val_accuracy: 0.931

In [ ]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 1s 5ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,536,0,1,0,0,0
SITTING,0,417,72,0,0,2
STANDING,0,108,424,0,0,0
WALKING,0,0,2,480,13,1
WALKING_DOWNSTAIRS,0,0,0,0,419,1
WALKING_UPSTAIRS,0,0,0,17,8,446


In [ ]:
score = model.evaluate(X_test, y_test)

print("\n   categorica_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 6ms/step - loss: 0.2493 - accuracy: 0.9237

   categorica_crossentropy  ||   accuracy 
  ____________________________________
[0.24925614893436432, 0.9236511588096619]


#### 2.2 Second Model for 2-Layer of LSTM with more hyperparameter tunning

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128, 64)           18944     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_4 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_4 (Dropout)         (None, 48)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 294       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 12s 17ms/step - loss: 1.6609 - accuracy: 0.7193 - val_loss: 1.4947 - val_accuracy: 0.7184
Epoch 2/30
460/460 [==============================] - 7s 14ms/step - loss: 0.9068 - accuracy: 0.8592 - val_loss: 0.7450 - val_accuracy: 0.8850
Epoch 3/30
460/460 [==============================] - 7s 15ms/step - loss: 0.6648 - accuracy: 0.9027 - val_loss: 0.5959 - val_accuracy: 0.9067
Epoch 4/30
460/460 [==============================] - 6s 13ms/step - loss: 0.5454 - accuracy: 0.9222 - val_loss: 0.5312 - val_accuracy: 0.9013
Epoch 5/30
460/460 [==============================] - 9s 19ms/step - loss: 0.4857 - accuracy: 0.9202 - val_loss: 0.4506 - val_accuracy: 0.9084
Epoch 6/30
460/460 [==============================] - 6s 14ms/step - loss: 0.4066 - accuracy: 0.9266 - val_loss: 0.4097 - val_accuracy: 0.9118
Epoch 7/30
460/460 [==============================] - 7s 15ms/step - loss: 0.3503 - accuracy: 0.9293 - val_loss: 0.4718 - val_accuracy: 0.899

In [ ]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 1s 6ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,7,396,86,0,0,2
STANDING,0,85,447,0,0,0
WALKING,0,0,0,487,2,7
WALKING_DOWNSTAIRS,0,0,0,0,418,2
WALKING_UPSTAIRS,0,0,0,0,22,449


In [ ]:
score = model.evaluate(X_test, y_test)

print("\n   categorical_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 8ms/step - loss: 0.2195 - accuracy: 0.9277

   categorical_crossentropy  ||   accuracy 
  ____________________________________
[0.21945692598819733, 0.9277231097221375]


In [ ]:
model.save("lstm_model.h5")

In [ ]:
# Load the LSTM model
from tensorflow.keras.models import load_model
model_1 = load_model('lstm_model.h5')

# Preprocess and prepare the input data
import numpy as np
data = np.load('input_data.npy')
data = preprocess_data(data)

# Convert the input data into a sequence format
seq_length = 10 # length of input sequence
num_features = 3 # number of input features
data_seq = []
for i in range(seq_length, len(data)):
    data_seq.append(data[i-seq_length:i, :])

# Feed the input data into the model
data_seq = np.array(data_seq)
output_seq = model.predict(data_seq)
# Display the output to the user
import matplotlib.pyplot as plt
plt.plot(output_seq)
plt.show()


FileNotFoundError: ignored